In [2]:
!pip install -U scikit-learn

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 2.8 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 8.9 MB/s eta 0:00:00:00:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Define TPU strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():

    # Load data
    df = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')
    X = df.drop(['label'], axis=1).values
    y = df['label'].values

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape data for use with VGG16
    X_train = X_train.reshape(-1, 64, 64)
    X_test = X_test.reshape(-1, 64, 64)
    X_train = np.repeat(X_train[..., np.newaxis], 3, -1)
    X_test = np.repeat(X_test[..., np.newaxis], 3, -1)

    # Load VGG16 model and freeze convolutional layers
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    vgg16.trainable = False

    # Create a CNN model
    model = Sequential()
    model.add(vgg16)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=250, validation_data=(X_test, y_test))

    # Perform predictions using the model
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    # Convert predicted labels back to original labels using label encoder
    y_pred_labels = label_encoder.inverse_transform(y_pred)
    
    # Print the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    print('Test Accuracy:', accuracy)


D0427 05:51:21.662438735      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0427 05:51:21.662470247      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0427 05:51:21.662473625      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0427 05:51:21.662476074      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0427 05:51:21.662478354      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0427 05:51:21.662480872      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0427 05:51:21.662484157      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0427 05:51:21.

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


...

313/313 [==============================] - ETA: 0s - loss: 4.9678 - accuracy: 0.0120

2023-04-27 05:52:23.064903: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-04-27 05:52:23.172822: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


313/313 [==============================] - 30s 69ms/step - loss: 4.9678 - accuracy: 0.0120 - val_loss: 4.6727 - val_accuracy: 0.0212
Epoch 2/250
313/313 [==============================] - 4s 13ms/step - loss: 4.6729 - accuracy: 0.0244 - val_loss: 4.6076 - val_accuracy: 0.0420
Epoch 3/250
313/313 [==============================] - 4s 13ms/step - loss: 4.5903 - accuracy: 0.0360 - val_loss: 4.5070 - val_accuracy: 0.0552
Epoch 4/250
313/313 [==============================] - 4s 13ms/step - loss: 4.4969 - accuracy: 0.0489 - val_loss: 4.3828 - val_accuracy: 0.0768
Epoch 5/250
313/313 [==============================] - 4s 13ms/step - loss: 4.3935 - accuracy: 0.0565 - val_loss: 4.2738 - val_accuracy: 0.0932
Epoch 6/250
313/313 [==============================] - 4s 14ms/step - loss: 4.2867 - accuracy: 0.0667 - val_loss: 4.1664 - val_accuracy: 0.1164
Epoch 7/250
313/313 [==============================] - 4s 13ms/step - loss: 4.1965 - accuracy: 0.0819 - val_loss: 4.0629 - val_accuracy: 0.1261
Epo

2023-04-27 06:09:43.867724: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-04-27 06:09:43.955627: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


79/79 [==============================] - 12s 75ms/step
Test Accuracy: 0.6374549819927972


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Define TPU strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():

    # Load data
    df = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')
    X = df.drop(['label'], axis=1).values
    y = df['label'].values

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape data for use with ResNet50
    X_train = X_train.reshape(-1, 64, 64)
    X_test = X_test.reshape(-1, 64, 64)
    X_train = np.repeat(X_train[..., np.newaxis], 3, -1)
    X_test = np.repeat(X_test[..., np.newaxis], 3, -1)

    # Load ResNet50 model and freeze convolutional layers
    resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    resnet50.trainable = False

    # Create a CNN model
    model = Sequential()
    model.add(resnet50)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(X_train, y_train, epochs=250, validation_data=(X_test, y_test))

    # Perform predictions using the CNN model
    cnn_y_pred = model.predict(X_test)

    # Train an XGBoost model
    xgb_model = xgb.XGBClassifier(objective='multi:softmax')
    xgb_model.fit(X_train.reshape(-1, 4096), y_train)
    
    # Perform predictions using the XGBoost model
    xgb_y_pred = xgb_model.predict(X_test.reshape(-1, 4096))

    # Combine the predictions from both models
    y_pred = np.argmax(cnn_y_pred, axis=1) + xgb_y_pred
    
    # Convert predicted labels back to original labels using label encoder
    y_pred_labels = label_encoder.inverse_transform(y_pred)

    # Print the accuracy score
    accuracy = accuracy_score(y_test, y_pred_labels)
    print('Test Accuracy:', accuracy)
    model.save('/kaggle/working/gait_classification_model.h5')


D0429 14:38:12.850430636      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0429 14:38:12.850477745      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0429 14:38:12.850482151      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0429 14:38:12.850485508      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0429 14:38:12.850488432      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0429 14:38:12.850491515      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0429 14:38:12.850494572      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0429 14:38:12.

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


...

313/313 [==============================] - ETA: 0s - loss: 4.7215 - accuracy: 0.0155

2023-04-29 14:39:36.059506: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-04-29 14:39:36.336440: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


313/313 [==============================] - 32s 65ms/step - loss: 4.7215 - accuracy: 0.0155 - val_loss: 4.5747 - val_accuracy: 0.0336
Epoch 2/250
313/313 [==============================] - 7s 21ms/step - loss: 4.5125 - accuracy: 0.0376 - val_loss: 4.3510 - val_accuracy: 0.0748
Epoch 3/250
313/313 [==============================] - 7s 21ms/step - loss: 4.3284 - accuracy: 0.0629 - val_loss: 4.1630 - val_accuracy: 0.1104
Epoch 4/250
313/313 [==============================] - 7s 21ms/step - loss: 4.1518 - accuracy: 0.0919 - val_loss: 3.9798 - val_accuracy: 0.1269
Epoch 5/250
313/313 [==============================] - 7s 21ms/step - loss: 4.0075 - accuracy: 0.1104 - val_loss: 3.8591 - val_accuracy: 0.1697
Epoch 6/250
313/313 [==============================] - 7s 21ms/step - loss: 3.8679 - accuracy: 0.1302 - val_loss: 3.7380 - val_accuracy: 0.1837
Epoch 7/250
313/313 [==============================] - 7s 22ms/step - loss: 3.7480 - accuracy: 0.1509 - val_loss: 3.6362 - val_accuracy: 0.2025
Epo

2023-04-29 15:07:37.414763: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-04-29 15:07:37.660842: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


79/79 [==============================] - 10s 51ms/step
Test Accuracy: 0.9324129651860744


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Define TPU strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():

    # Load data
    df = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')
    X = df.drop(['label'], axis=1).values
    y = df['label'].values

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape data for use with ResNet50
    X_train = X_train.reshape(-1, 64, 64)
    X_test = X_test.reshape(-1, 64, 64)
    X_train = np.repeat(X_train[..., np.newaxis], 3, -1)
    X_test = np.repeat(X_test[..., np.newaxis], 3, -1)

    # Load ResNet50 model and freeze convolutional layers
    resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    resnet50.trainable = False

    # Create a CNN model
    model = Sequential()
    model.add(resnet50)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=250, validation_data=(X_test, y_test))

    # Perform predictions using the model
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    # Convert predicted labels back to original labels using label encoder
    y_pred_labels = label_encoder.inverse_transform(y_pred)

    # Print the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    print('Test Accuracy:', accuracy) 
    # Save the model
    model.save('/kaggle/working/gait_classification_model.h5')

